In [17]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/HdlDYng8g9s" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

In [10]:
from sklearn import svm, datasets
iris = datasets.load_iris()


In [13]:
import pandas as pd
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df['flower'] = iris.target
df['flower'].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,0
48,5.3,3.7,1.5,0.2,0
49,5.0,3.3,1.4,0.2,0
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(iris.data, iris.target, test_size = 0.3)

In [15]:
from sklearn.model_selection import cross_val_score

In [16]:
import numpy as np
kernels = ['rbf', 'linear']
C = [1, 10, 20]
avg_scores = {}

for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel = kval, C = cval, gamma='auto'), 
                                    iris.data, iris.target, cv = 5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)
        
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

## Hyper parameter search using GridSearchCV

In [18]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(svm.SVC(gamma = 'auto'), {
'C' : [1, 10, 20],
'kernel': ['rbf', 'linear']
}, cv = 5, return_train_score=False)

clf.fit(iris.data, iris.target)
clf.cv_results_


{'mean_fit_time': array([0.00158896, 0.00120215, 0.00185833, 0.00119653, 0.00118837,
        0.00099707]),
 'std_fit_time': array([0.00080042, 0.00039566, 0.00085111, 0.00039833, 0.00038667,
        0.00063053]),
 'mean_score_time': array([0.00099487, 0.00019965, 0.00115824, 0.00060139, 0.00019884,
        0.00059419]),
 'std_score_time': array([0.00063091, 0.0003993 , 0.00070888, 0.00049106, 0.00039768,
        0.00048521]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [21]:
df= pd.DataFrame(clf.cv_results_)
df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [22]:
clf.best_score_

0.9800000000000001

In [23]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [29]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma = 'auto'), {
'C' : [1, 10, 20],
'kernel': ['rbf', 'linear']
},
cv = 5,
return_train_score = False,
n_iter = 2
)

rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,linear,0.98
1,10,rbf,0.98


## How to choose the best model/classifier

In [30]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [32]:
model_params = {
'svm':{
'model': svm.SVC(gamma = 'auto'),
    'params': {
    'C' : [1, 10, 20],
    'kernel': ['rbf', 'linear']
    }
},
'random_forest': {
'model': RandomForestClassifier(),
'params': {
'n_estimators': [1, 5, 10]
}
},

'logistic_regression': {
'model': LogisticRegression(solver = 'liblinear', multi_class='auto'),
'params': {
'C': [1, 5, 10]
}
}
}


In [33]:
scores = []
for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv = 5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
    'model' : model_name,
    'best_score' : clf.best_score_,
    'best_params' : clf.best_params_
    })

In [34]:
df=  pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
